# Mount Device

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download Packages

In [2]:
# or clone this repo, removing the '-' to allow python imports:
!git clone https://github.com/timesler/facenet-pytorch.git facenet_pytorch
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install mxnet
!pip install torchmetrics

fatal: destination path 'facenet_pytorch' already exists and is not an empty directory.
Looking in indexes: https://download.pytorch.org/whl/cu118


# Import dependencies

In [3]:
import numpy as np
import os
np.bool = bool
import mxnet as mx
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import math
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from facenet_pytorch import MTCNN
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from torchmetrics import Accuracy
import torch_xla.utils.serialization as xser
from Utils import preprocess_image, CANONICAL_LANDMARKS
from CasiaWebFace import CASIAWebFaceDataset
from Intermediate_Strategy import MobileFaceNetIntermediate
from ArcFace import ArcFaceLoss
from MobileFaceNet import MobileFaceNet
from train_functions import train, train_tpu

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/content/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless

In [ ]:
def _mp_fn(rank, flags, rec_file, idx_file):
    # Define the device as TPU
    device = xm.xla_device()

    # Define the mean and std for normalization
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # Define a named function to replace the lambda (already provided earlier)
    def preprocess_and_transform(img):
        img = preprocess_image(img)  # Ensure preprocess_image is defined elsewhere
        img = transforms.ToTensor()(img)
        img = transforms.Normalize(mean=mean, std=std)(img)
        return img

    # Define the transformation pipeline for training with augmentation
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),  # Rotate by +/- 10 degrees
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomResizedCrop(112, scale=(0.8, 1.0)),  # Randomly crop and resize to 112x112
        transforms.Lambda(preprocess_and_transform)
    ])

    # Initialize the dataset inside the function
    full_dataset = CASIAWebFaceDataset(rec_file, idx_file, transform=None)  # No transform applied initially

    # Apply the transformation to the full dataset
    full_dataset.transform = train_transform

    BATCH_SIZE = 128
    # Create a DataLoader for the full dataset
    full_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, num_workers=8, shuffle=True)

    # Wrap DataLoader with MpDeviceLoader for TPU distributed loading
    train_loader = pl.MpDeviceLoader(full_loader, device)

    # Initialize model, loss, optimizer, etc.
    model = MobileFaceNetIntermediate(embedding_size=128).to(device)
    arc_face = ArcFaceLoss(num_classes=full_dataset.get_num_unique_labels(), embedding_size=128, margin=0.3, scale=16, device=device).to(device)
    # Define the optimizer
    optimizer = torch.optim.SGD(
        [
            {'params': model.conv1.parameters()},
            {'params': model.dwconv2.parameters()},
            {'params': model.blocks.parameters()},
            {'params': model.gdconv.parameters()},
            {'params': model.fc.parameters(), 'weight_decay': 0.01},
            {'params': arc_face.parameters(), 'weight_decay': 0.01}
        ],
        lr=0.0001,
        momentum=0.9,
        weight_decay=0.00001,
        nesterov=True
    )
    model_path = "/content/nModel.pth"
    if os.path.exists(model_path):
      checkpoint = torch.load(model_path)
      model.load_state_dict(checkpoint['model_state_dict'], strict=False)
      arc_face.load_state_dict(checkpoint['arcface_state_dict'], strict=False)
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      print("Checkpoint loaded successfully!")

    # Define the learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

    # Define accuracy metrics (optional, you can modify or remove this if not needed)
    train_accuracy = Accuracy(task='multiclass', num_classes=full_dataset.get_num_unique_labels()).to(device)

    # Call the training function without validation or testing
    epoch_losses, train_accuracies = train_tpu(
        model, train_accuracy, device, optimizer, arc_face, scheduler, train_loader
    )
# Define any additional flags or arguments
FLAGS = {}

# Define file paths (these should be passed to the function)
rec_file = '/content/drive/MyDrive/casia-webface/train.rec'
idx_file = '/content/drive/MyDrive/casia-webface/train.idx'

# Spawn the training across 8 TPU cores
xmp.spawn(_mp_fn, args=(FLAGS, rec_file, idx_file), nprocs=1, start_method='fork')


<ipython-input-4-5d8291e3ca56>:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


Checkpoint loaded successfully!
Epoch 1/75, Current LR: 0.000010
Train Loss: 4.7499, Train Accuracy: 0.3392
Epoch 2/75, Current LR: 0.000010
Train Loss: 4.7433, Train Accuracy: 0.3405
Epoch 3/75, Current LR: 0.000010
Train Loss: 4.7376, Train Accuracy: 0.3415
